In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as py

In [2]:
import os
from os import listdir as ls

In [3]:
ls("/home/DAVIDSON/mawang1/Workspace/CSC381/implementation-project-dl-f22-hueynataliemaureenilina/")

['ParticipantsResults2012',
 'evaluationTools',
 'CROHME_papers',
 'README.md',
 'index.html',
 'CROHME2011_data',
 'CROHME_full_v2.zip',
 'crohme.css',
 'CROHME2013_data',
 '.git',
 '.ipynb_checkpoints',
 'readme.txt',
 'scratchwork.ipynb',
 'CROHME2012_data']

In [ ]:
# path_zip = '/home/DAVIDSON/mawang1/Workspace/CSC381/implementation-project-dl-f22-hueynataliemaureenilina/'

In [ ]:
import os
import zipfile
# with zipfile.ZipFile('CROHME_full_v2.zip', 'r') as zip_ref:
    # zip_ref.extractall(path_zip)

In [4]:
path_2013 = '/home/DAVIDSON/mawang1/Workspace/CSC381/implementation-project-dl-f22-hueynataliemaureenilina/CROHME2013_data/TrainINKML/'

In [ ]:
path_Mathbrush = path_2013 + 'MathBrush/'

In [ ]:
ls(path_Mathbrush)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xml.etree.ElementTree as ET
import os
import numpy as np
import pandas as pd

In [ ]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
SEED = 999
seed_everything(SEED)